In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
sns.set_style('whitegrid')
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
pd.set_option('display.width', 200)
pd.set_option('display.colheader_justify', 'center')
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [2]:
# Load the dataset
df = pd.read_excel('_data/_matches_list_v0.xlsx')

In [3]:
##### REMOVE UNPLAYED & UNFINISHED MATCHES #####

# Filter out rows where 'Comment' is either 'Not Played' or 'Unfinished'
print(df.shape)
df = df[(df['Comment'] != 'Not Played') & (df['Comment'] != 'Unfinished')]
df.reset_index(drop=True, inplace=True)

# Drop the 'Comment' column
df.drop('Comment', axis=1, inplace=True)

print(df.shape)
df.head()

(35447, 105)
(34121, 104)


,match_id,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,Year,match_num,winner_id,winner_seed,winner_entry,winner_name,winner_hand,winner_ht,winner_ioc,winner_age,loser_id,loser_seed,loser_entry,loser_name,loser_hand,loser_ht,loser_ioc,loser_age,score,best_of,round,minutes,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points,ATP,Location,Tournament,Date,Series,Court,Surface,Round,Best of,Winner,Loser,WRank,LRank,WPts,LPts,W1,L1,W2,L2,W3,L3,W4,L4,W5,L5,Wsets,Lsets,B365W,B365L,EXW,EXL,LBW,LBL,PSW,PSL,SJW,SJL,MaxW,MaxL,AvgW,AvgL,UBW,UBL,CBW,CBL,IWW,IWL,SBW,SBL,B&WW,B&WL,GBW,GBL
0,2009-339-0011,2009-339,Brisbane,Hard,32,A,2009-01-04,2009,11,103898,NaN,NaN,Julien Benneteau,R,185.00,FRA,27.00,104068,NaN,NaN,Robby Ginepri,R,183.00,USA,26.20,3-6 6-3 7-5,3,R32,127.00,10.00,9.00,99.00,62.00,41.00,17.00,15.00,2.00,6.00,8.00,2.00,88.00,53.00,32.00,21.00,15.00,4.00,9.00,40.00,1608.00,49.00,1480.00,1,Brisbane,Brisbane International,2009-01-04,ATP250,Outdoor,Hard,1st Round,3.00,Benneteau J.,Ginepri R.,40.00,49.00,1608.00,1480.00,3.00,6.00,6.00,3.00,7.00,5.00,NaN,NaN,NaN,NaN,2.00,1.00,2.10,1.66,2.25,1.65,2.10,1.66,NaN,NaN,2.10,1.73,NaN,NaN,NaN,NaN,1.96,1.80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2009-339-0012,2009-339,Brisbane,Hard,32,A,2009-01-04,2009,12,104417,4.00,NaN,Robin Soderling,R,193.00,SWE,24.30,105023,NaN,NaN,Sam Querrey,R,198.00,USA,21.20,6-3 6-3,3,R32,55.00,7.00,2.00,52.00,30.00,26.00,13.00,9.00,0.00,0.00,4.00,3.00,49.00,27.00,20.00,9.00,9.00,2.00,5.00,17.00,2650.00,36.00,1740.00,1,Brisbane,Brisbane International,2009-01-04,ATP250,Outdoor,Hard,1st Round,3.00,Soderling R.,Querrey S.,17.00,36.00,2650.00,1740.00,6.00,3.00,6.00,3.00,NaN,NaN,NaN,NaN,NaN,NaN,2.00,0.00,1.44,2.62,1.49,2.65,1.53,2.37,NaN,NaN,1.50,2.62,NaN,NaN,NaN,NaN,1.40,2.85,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2009-339-0013,2009-339,Brisbane,Hard,32,A,2009-01-04,2009,13,104755,7.00,NaN,Richard Gasquet,R,185.00,FRA,22.50,102967,NaN,NaN,Marc Gicquel,R,188.00,FRA,31.70,4-6 6-2 6-2,3,R32,133.00,6.00,5.00,91.00,42.00,29.00,26.00,13.00,3.00,5.00,7.00,6.00,100.00,45.00,30.00,23.00,13.00,13.00,18.00,23.00,2320.00,50.00,1428.00,1,Brisbane,Brisbane International,2009-01-04,ATP250,Outdoor,Hard,1st Round,3.00,Gasquet R.,Gicquel M.,23.00,50.00,2320.00,1428.00,4.00,6.00,6.00,2.00,6.00,2.00,NaN,NaN,NaN,NaN,2.00,1.00,1.25,3.75,1.27,3.81,1.28,3.50,NaN,NaN,1.25,3.50,NaN,NaN,NaN,NaN,1.27,3.60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2009-339-0014,2009-339,Brisbane,Hard,32,A,2009-01-04,2009,14,103758,NaN,NaN,Taylor Dent,R,188.00,USA,27.70,104327,NaN,NaN,Steve Darcis,R,178.00,BEL,24.80,7-6(0) 6-2,3,R32,86.00,8.00,5.00,68.00,37.00,31.00,13.00,10.00,6.00,8.00,4.00,2.00,70.00,46.00,24.00,14.00,10.00,1.00,5.00,865.00,28.00,59.00,1362.00,1,Brisbane,Brisbane International,2009-01-04,ATP250,Outdoor,Hard,1st Round,3.00,Dent T.,Darcis S.,865.00,59.00,28.00,1362.00,7.00,6.00,6.00,2.00,NaN,NaN,NaN,NaN,NaN,NaN,2.00,0.00,2.75,1.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.00,1.37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2009-339-0005,2009-339,Brisbane,Hard,32,A,2009-01-04,2009,5,104269,3.00,NaN,Fernando Verdasco,L,188.00,ESP,25.10,106071,NaN,WC,Bernard Tomic,R,193.00,AUS,16.20,6-4 6-2,3,R32,65.00,1.00,2.00,50.00,39.00,31.00,5.00,9.00,3.00,4.00,3.00,1.00,56.00,34.00,19.00,9.00,9.00,9.00,13.00,15.00,2830.00,764.00,42.00,1,Brisbane,Brisbane International,2009-01-05,ATP250,Outdoor,Hard,1st Round,3.00,Verdasco F.,Tomic B.,15.00,764.00,2830.00,42.00,6.00,4.00,6.00,2.00,NaN,NaN,NaN,NaN,NaN,NaN,2.00,0.00,1.06,8.00,1.06,9.52,1.04,9.00,NaN,NaN,1.03,11.00,NaN,NaN,NaN,NaN,1.04,11.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
##### CREATE POINTS COLUMNS #####

# Create the total_points, winner_points, loser_points columns
w_2ndIn = df['w_svpt'] - df['w_1stIn'] - df['w_df']
l_2ndIn = df['l_svpt'] - df['l_1stIn'] - df['l_df']

df['winner_points'] = df['w_1stWon'] + df['w_2ndWon'] + (df['l_1stIn'] - df['l_1stWon']) + (l_2ndIn - df['l_2ndWon']) + df['l_df']
df['loser_points'] = df['l_1stWon'] + df['l_2ndWon'] + (df['w_1stIn'] - df['w_1stWon']) + (w_2ndIn - df['w_2ndWon']) + df['w_df']
df['total_points'] = df['winner_points'] + df['loser_points']

df.head()

,match_id,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,Year,match_num,winner_id,winner_seed,winner_entry,winner_name,winner_hand,winner_ht,winner_ioc,winner_age,loser_id,loser_seed,loser_entry,loser_name,loser_hand,loser_ht,loser_ioc,loser_age,score,best_of,round,minutes,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points,ATP,Location,Tournament,Date,Series,Court,Surface,Round,Best of,Winner,Loser,WRank,LRank,WPts,LPts,W1,L1,W2,L2,W3,L3,W4,L4,W5,L5,Wsets,Lsets,B365W,B365L,EXW,EXL,LBW,LBL,PSW,PSL,SJW,SJL,MaxW,MaxL,AvgW,AvgL,UBW,UBL,CBW,CBL,IWW,IWL,SBW,SBL,B&WW,B&WL,GBW,GBL,winner_points,loser_points,total_points
0,2009-339-0011,2009-339,Brisbane,Hard,32,A,2009-01-04,2009,11,103898,NaN,NaN,Julien Benneteau,R,185.00,FRA,27.00,104068,NaN,NaN,Robby Ginepri,R,183.00,USA,26.20,3-6 6-3 7-5,3,R32,127.00,10.00,9.00,99.00,62.00,41.00,17.00,15.00,2.00,6.00,8.00,2.00,88.00,53.00,32.00,21.00,15.00,4.00,9.00,40.00,1608.00,49.00,1480.00,1,Brisbane,Brisbane International,2009-01-04,ATP250,Outdoor,Hard,1st Round,3.00,Benneteau J.,Ginepri R.,40.00,49.00,1608.00,1480.00,3.00,6.00,6.00,3.00,7.00,5.00,NaN,NaN,NaN,NaN,2.00,1.00,2.10,1.66,2.25,1.65,2.10,1.66,NaN,NaN,2.10,1.73,NaN,NaN,NaN,NaN,1.96,1.80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,93.00,94.00,187.00
1,2009-339-0012,2009-339,Brisbane,Hard,32,A,2009-01-04,2009,12,104417,4.00,NaN,Robin Soderling,R,193.00,SWE,24.30,105023,NaN,NaN,Sam Querrey,R,198.00,USA,21.20,6-3 6-3,3,R32,55.00,7.00,2.00,52.00,30.00,26.00,13.00,9.00,0.00,0.00,4.00,3.00,49.00,27.00,20.00,9.00,9.00,2.00,5.00,17.00,2650.00,36.00,1740.00,1,Brisbane,Brisbane International,2009-01-04,ATP250,Outdoor,Hard,1st Round,3.00,Soderling R.,Querrey S.,17.00,36.00,2650.00,1740.00,6.00,3.00,6.00,3.00,NaN,NaN,NaN,NaN,NaN,NaN,2.00,0.00,1.44,2.62,1.49,2.65,1.53,2.37,NaN,NaN,1.50,2.62,NaN,NaN,NaN,NaN,1.40,2.85,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59.00,42.00,101.00
2,2009-339-0013,2009-339,Brisbane,Hard,32,A,2009-01-04,2009,13,104755,7.00,NaN,Richard Gasquet,R,185.00,FRA,22.50,102967,NaN,NaN,Marc Gicquel,R,188.00,FRA,31.70,4-6 6-2 6-2,3,R32,133.00,6.00,5.00,91.00,42.00,29.00,26.00,13.00,3.00,5.00,7.00,6.00,100.00,45.00,30.00,23.00,13.00,13.00,18.00,23.00,2320.00,50.00,1428.00,1,Brisbane,Brisbane International,2009-01-04,ATP250,Outdoor,Hard,1st Round,3.00,Gasquet R.,Gicquel M.,23.00,50.00,2320.00,1428.00,4.00,6.00,6.00,2.00,6.00,2.00,NaN,NaN,NaN,NaN,2.00,1.00,1.25,3.75,1.27,3.81,1.28,3.50,NaN,NaN,1.25,3.50,NaN,NaN,NaN,NaN,1.27,3.60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,102.00,89.00,191.00
3,2009-339-0014,2009-339,Brisbane,Hard,32,A,2009-01-04,2009,14,103758,NaN,NaN,Taylor Dent,R,188.00,USA,27.70,104327,NaN,NaN,Steve Darcis,R,178.00,BEL,24.80,7-6(0) 6-2,3,R32,86.00,8.00,5.00,68.00,37.00,31.00,13.00,10.00,6.00,8.00,4.00,2.00,70.00,46.00,24.00,14.00,10.00,1.00,5.00,865.00,28.00,59.00,1362.00,1,Brisbane,Brisbane International,2009-01-04,ATP250,Outdoor,Hard,1st Round,3.00,Dent T.,Darcis S.,865.00,59.00,28.00,1362.00,7.00,6.00,6.00,2.00,NaN,NaN,NaN,NaN,NaN,NaN,2.00,0.00,2.75,1.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.00,1.37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,76.00,62.00,138.00
4,2009-339-0005,2009-339,Brisbane,Hard,32,A,2009-01-04,2009,5,104269,3.00,NaN,Fernando Verdasco,L,188.00,ESP,25.10,106071,NaN,WC,Bernard Tomic,R,193.00,AUS,16.20,6-4 6-2,3,R32,65.00,1.00,2.00,50.00,39.00,31.00,5.00,9.00,3.00,4.00,3.00,1.00,56.00,34.00,19.00,9.00,9.00,9.00,13.00,15.00,2830.00,764.00,42.00,1,Brisbane,Brisbane International,2009-01-05,ATP250,Outdoor,Hard,1st Round,3.00,Verdasco F.,Tomic B.,15.00,764.00,2830.00,42.00,6.00,4.00,6.00,2.00,NaN,NaN,NaN,NaN,NaN,NaN,2.00,0.00,1.06,8.00,1.06,9.52,1.04,9.00,NaN,NaN,1.03,11.00,NaN,NaN,NaN,NaN,1.04,11.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64.00,42.00,106.00


In [5]:
##### CREATE GAMES COLUMNS #####

# Create the total_games, winner_games, loser_games columns
def calculate_winner_games(score):
    games = score.split()
    games_won = 0
    for game_score in games:
        # Split the score into winner and loser points
        points = [s.split('(')[0] for s in game_score.split("-")]
        # Check if the score is valid (it could be 'W/O' or 'DEF', etc.)
        if all(point.isdigit() for point in points):
            # Add the winner's games to the total
            games_won += int(points[0])
    return games_won

def calculate_loser_games(score):
    games = score.split()
    games_lost = 0
    for game_score in games:
        # Split the score into winner and loser points
        points = [s.split('(')[0] for s in game_score.split("-")]
        # Check if the score is valid (it could be 'W/O' or 'DEF', etc.)
        if all(point.isdigit() for point in points) and len(points) == 2:
            # Add the loser's games to the total
            games_lost += int(points[1])
    return games_lost

# Apply the functions to the 'score' column
df['total_games'] = df['score'].apply(calculate_winner_games) + df['score'].apply(calculate_loser_games)
df['winner_games'] = df['score'].apply(calculate_winner_games)
df['loser_games'] = df['score'].apply(calculate_loser_games)


# Create winner_sets and loser_sets columns
def calculate_winner_sets(score):
    sets = score.split()
    sets_won = 0
    for set_score in sets:
        # Split the score into winner and loser points
        points = [s.split('(')[0] for s in set_score.split("-")]
        # Check if the score is valid (it could be 'W/O' or 'DEF', etc.)
        if all(point.isdigit() for point in points):
            # If the winner's points are greater than the loser's, increment sets_won
            if int(points[0]) > int(points[1]):
                sets_won += 1
    return sets_won

def calculate_loser_sets(score):
    """Calculate the number of sets won by the loser from the score."""
    sets = score.split()
    loser_sets = 0
    for set_score in sets:
        scores = [int(s.split('(')[0]) for s in set_score.split("-")]
        if len(scores) == 2 and scores[0] < scores[1]:
            loser_sets += 1
    return loser_sets

# Apply the function to the 'score' column
df['winner_sets'] = df['score'].apply(calculate_winner_sets)
df['loser_sets'] = df['score'].apply(calculate_loser_sets)

df.head()

,match_id,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,Year,match_num,winner_id,winner_seed,winner_entry,winner_name,winner_hand,winner_ht,winner_ioc,winner_age,loser_id,loser_seed,loser_entry,loser_name,loser_hand,loser_ht,loser_ioc,loser_age,score,best_of,round,minutes,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points,ATP,Location,Tournament,Date,Series,Court,Surface,Round,Best of,Winner,Loser,WRank,LRank,WPts,LPts,W1,L1,W2,L2,W3,L3,W4,L4,W5,L5,Wsets,Lsets,B365W,B365L,EXW,EXL,LBW,LBL,PSW,PSL,SJW,SJL,MaxW,MaxL,AvgW,AvgL,UBW,UBL,CBW,CBL,IWW,IWL,SBW,SBL,B&WW,B&WL,GBW,GBL,winner_points,loser_points,total_points,total_games,winner_games,loser_games,winner_sets,loser_sets
0,2009-339-0011,2009-339,Brisbane,Hard,32,A,2009-01-04,2009,11,103898,NaN,NaN,Julien Benneteau,R,185.00,FRA,27.00,104068,NaN,NaN,Robby Ginepri,R,183.00,USA,26.20,3-6 6-3 7-5,3,R32,127.00,10.00,9.00,99.00,62.00,41.00,17.00,15.00,2.00,6.00,8.00,2.00,88.00,53.00,32.00,21.00,15.00,4.00,9.00,40.00,1608.00,49.00,1480.00,1,Brisbane,Brisbane International,2009-01-04,ATP250,Outdoor,Hard,1st Round,3.00,Benneteau J.,Ginepri R.,40.00,49.00,1608.00,1480.00,3.00,6.00,6.00,3.00,7.00,5.00,NaN,NaN,NaN,NaN,2.00,1.00,2.10,1.66,2.25,1.65,2.10,1.66,NaN,NaN,2.10,1.73,NaN,NaN,NaN,NaN,1.96,1.80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,93.00,94.00,187.00,30,16,14,2,1
1,2009-339-0012,2009-339,Brisbane,Hard,32,A,2009-01-04,2009,12,104417,4.00,NaN,Robin Soderling,R,193.00,SWE,24.30,105023,NaN,NaN,Sam Querrey,R,198.00,USA,21.20,6-3 6-3,3,R32,55.00,7.00,2.00,52.00,30.00,26.00,13.00,9.00,0.00,0.00,4.00,3.00,49.00,27.00,20.00,9.00,9.00,2.00,5.00,17.00,2650.00,36.00,1740.00,1,Brisbane,Brisbane International,2009-01-04,ATP250,Outdoor,Hard,1st Round,3.00,Soderling R.,Querrey S.,17.00,36.00,2650.00,1740.00,6.00,3.00,6.00,3.00,NaN,NaN,NaN,NaN,NaN,NaN,2.00,0.00,1.44,2.62,1.49,2.65,1.53,2.37,NaN,NaN,1.50,2.62,NaN,NaN,NaN,NaN,1.40,2.85,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59.00,42.00,101.00,18,12,6,2,0
2,2009-339-0013,2009-339,Brisbane,Hard,32,A,2009-01-04,2009,13,104755,7.00,NaN,Richard Gasquet,R,185.00,FRA,22.50,102967,NaN,NaN,Marc Gicquel,R,188.00,FRA,31.70,4-6 6-2 6-2,3,R32,133.00,6.00,5.00,91.00,42.00,29.00,26.00,13.00,3.00,5.00,7.00,6.00,100.00,45.00,30.00,23.00,13.00,13.00,18.00,23.00,2320.00,50.00,1428.00,1,Brisbane,Brisbane International,2009-01-04,ATP250,Outdoor,Hard,1st Round,3.00,Gasquet R.,Gicquel M.,23.00,50.00,2320.00,1428.00,4.00,6.00,6.00,2.00,6.00,2.00,NaN,NaN,NaN,NaN,2.00,1.00,1.25,3.75,1.27,3.81,1.28,3.50,NaN,NaN,1.25,3.50,NaN,NaN,NaN,NaN,1.27,3.60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,102.00,89.00,191.00,26,16,10,2,1
3,2009-339-0014,2009-339,Brisbane,Hard,32,A,2009-01-04,2009,14,103758,NaN,NaN,Taylor Dent,R,188.00,USA,27.70,104327,NaN,NaN,Steve Darcis,R,178.00,BEL,24.80,7-6(0) 6-2,3,R32,86.00,8.00,5.00,68.00,37.00,31.00,13.00,10.00,6.00,8.00,4.00,2.00,70.00,46.00,24.00,14.00,10.00,1.00,5.00,865.00,28.00,59.00,1362.00,1,Brisbane,Brisbane International,2009-01-04,ATP250,Outdoor,Hard,1st Round,3.00,Dent T.,Darcis S.,865.00,59.00,28.00,1362.00,7.00,6.00,6.00,2.00,NaN,NaN,NaN,NaN,NaN,NaN,2.00,0.00,2.75,1.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.00,1.37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,76.00,62.00,138.00,21,13,8,2,0
4,2009-339-0005,2009-339,Brisbane,Hard,32,A,2009-01-04,2009,5,104269,3.00,NaN,Fernando Verdasco,L,188.00,ESP,25.10,106071,NaN,WC,Bernard Tomic,R,193.00,AUS,16.20,6-4 6-2,3,R32,65.00,1.00,2.00,50.00,39.00,31.00,5.00,9.00,3.00,4.00,3.00,1.00,56.00,34.00,19.00,9.00,9.00,9.00,13.00,15.00,2830.00,764.00,42.00,1,Brisbane,Brisbane International,2009-01-05,ATP250,Outdoor,Hard,1st Round,3.00,Verdasco F.,Tomic B.,15.00,764.00,2830.00,42.00,6.00,4.00,6.00,2.00,NaN,NaN,NaN,NaN,NaN,NaN,2.00,0.00,1.06,8.00,1.06,9.52,1.04,9.00,NaN,NaN,1.03,11.00,NaN,NaN,NaN,NaN,1.04,11.00,NaN,NaN,NaN,NaN,NaN,N

In [6]:
##### CONVERT COLUMN NAMES #####

# Define the new column names
new_column_names = {
    'Date': 'match_date',
    'Year': 'year',
    'Court': 'court',
    'winner_rank_points': 'winner_rankpt',
    'loser_rank_points': 'loser_rankpt',
    'B365W': 'winner_B365',
    'B365L': 'loser_B365',
    'EXW': 'winner_EX',
    'EXL': 'loser_EX',
    'LBW': 'winner_LB',
    'LBL': 'loser_LB',
    'PSW': 'winner_PS',
    'PSL': 'loser_PS',
    'SJW': 'winner_SJ',
    'SJL': 'loser_SJ',
    'UBW': 'winner_UB',
    'UBL': 'loser_UB',
    'MaxW': 'winner_Max',
    'MaxL': 'loser_Max',
    'AvgW': 'winner_Avg',
    'AvgL': 'loser_Avg'
}

# Update the column names
df.rename(columns=new_column_names, inplace=True)

df.head()

,match_id,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,year,match_num,winner_id,winner_seed,winner_entry,winner_name,winner_hand,winner_ht,winner_ioc,winner_age,loser_id,loser_seed,loser_entry,loser_name,loser_hand,loser_ht,loser_ioc,loser_age,score,best_of,round,minutes,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rankpt,loser_rank,loser_rankpt,ATP,Location,Tournament,match_date,Series,court,Surface,Round,Best of,Winner,Loser,WRank,LRank,WPts,LPts,W1,L1,W2,L2,W3,L3,W4,L4,W5,L5,Wsets,Lsets,winner_B365,loser_B365,winner_EX,loser_EX,winner_LB,loser_LB,winner_PS,loser_PS,winner_SJ,loser_SJ,winner_Max,loser_Max,winner_Avg,loser_Avg,winner_UB,loser_UB,CBW,CBL,IWW,IWL,SBW,SBL,B&WW,B&WL,GBW,GBL,winner_points,loser_points,total_points,total_games,winner_games,loser_games,winner_sets,loser_sets
0,2009-339-0011,2009-339,Brisbane,Hard,32,A,2009-01-04,2009,11,103898,NaN,NaN,Julien Benneteau,R,185.00,FRA,27.00,104068,NaN,NaN,Robby Ginepri,R,183.00,USA,26.20,3-6 6-3 7-5,3,R32,127.00,10.00,9.00,99.00,62.00,41.00,17.00,15.00,2.00,6.00,8.00,2.00,88.00,53.00,32.00,21.00,15.00,4.00,9.00,40.00,1608.00,49.00,1480.00,1,Brisbane,Brisbane International,2009-01-04,ATP250,Outdoor,Hard,1st Round,3.00,Benneteau J.,Ginepri R.,40.00,49.00,1608.00,1480.00,3.00,6.00,6.00,3.00,7.00,5.00,NaN,NaN,NaN,NaN,2.00,1.00,2.10,1.66,2.25,1.65,2.10,1.66,NaN,NaN,2.10,1.73,NaN,NaN,NaN,NaN,1.96,1.80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,93.00,94.00,187.00,30,16,14,2,1
1,2009-339-0012,2009-339,Brisbane,Hard,32,A,2009-01-04,2009,12,104417,4.00,NaN,Robin Soderling,R,193.00,SWE,24.30,105023,NaN,NaN,Sam Querrey,R,198.00,USA,21.20,6-3 6-3,3,R32,55.00,7.00,2.00,52.00,30.00,26.00,13.00,9.00,0.00,0.00,4.00,3.00,49.00,27.00,20.00,9.00,9.00,2.00,5.00,17.00,2650.00,36.00,1740.00,1,Brisbane,Brisbane International,2009-01-04,ATP250,Outdoor,Hard,1st Round,3.00,Soderling R.,Querrey S.,17.00,36.00,2650.00,1740.00,6.00,3.00,6.00,3.00,NaN,NaN,NaN,NaN,NaN,NaN,2.00,0.00,1.44,2.62,1.49,2.65,1.53,2.37,NaN,NaN,1.50,2.62,NaN,NaN,NaN,NaN,1.40,2.85,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59.00,42.00,101.00,18,12,6,2,0
2,2009-339-0013,2009-339,Brisbane,Hard,32,A,2009-01-04,2009,13,104755,7.00,NaN,Richard Gasquet,R,185.00,FRA,22.50,102967,NaN,NaN,Marc Gicquel,R,188.00,FRA,31.70,4-6 6-2 6-2,3,R32,133.00,6.00,5.00,91.00,42.00,29.00,26.00,13.00,3.00,5.00,7.00,6.00,100.00,45.00,30.00,23.00,13.00,13.00,18.00,23.00,2320.00,50.00,1428.00,1,Brisbane,Brisbane International,2009-01-04,ATP250,Outdoor,Hard,1st Round,3.00,Gasquet R.,Gicquel M.,23.00,50.00,2320.00,1428.00,4.00,6.00,6.00,2.00,6.00,2.00,NaN,NaN,NaN,NaN,2.00,1.00,1.25,3.75,1.27,3.81,1.28,3.50,NaN,NaN,1.25,3.50,NaN,NaN,NaN,NaN,1.27,3.60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,102.00,89.00,191.00,26,16,10,2,1
3,2009-339-0014,2009-339,Brisbane,Hard,32,A,2009-01-04,2009,14,103758,NaN,NaN,Taylor Dent,R,188.00,USA,27.70,104327,NaN,NaN,Steve Darcis,R,178.00,BEL,24.80,7-6(0) 6-2,3,R32,86.00,8.00,5.00,68.00,37.00,31.00,13.00,10.00,6.00,8.00,4.00,2.00,70.00,46.00,24.00,14.00,10.00,1.00,5.00,865.00,28.00,59.00,1362.00,1,Brisbane,Brisbane International,2009-01-04,ATP250,Outdoor,Hard,1st Round,3.00,Dent T.,Darcis S.,865.00,59.00,28.00,1362.00,7.00,6.00,6.00,2.00,NaN,NaN,NaN,NaN,NaN,NaN,2.00,0.00,2.75,1.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.00,1.37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,76.00,62.00,138.00,21,13,8,2,0
4,2009-339-0005,2009-339,Brisbane,Hard,32,A,2009-01-04,2009,5,104269,3.00,NaN,Fernando Verdasco,L,188.00,ESP,25.10,106071,NaN,WC,Bernard Tomic,R,193.00,AUS,16.20,6-4 6-2,3,R32,65.00,1.00,2.00,50.00,39.00,31.00,5.00,9.00,3.00,4.00,3.00,1.00,56.00,34.00,19.00,9.00,9.00,9.00,13.00,15.00,2830.00,764.00,42.00,1,Brisbane,Brisbane International,2009-01-05,ATP250,Outdoor,Hard,1st Round,3.00,Verdasco F.,Tomic B.,15.00,764.00,2830.00,42.00,6.00,4.00,6.00,2.00,NaN,NaN,NaN,NaN,NaN,NaN,2.00,0.00,1.06,8.00,1.0

In [7]:
##### CONVERT WINNER/LOSER COLUMNS #####

# Columns starting with 'winner_' and 'loser_'
winner_cols = df.columns[df.columns.str.startswith('winner_')]
loser_cols = df.columns[df.columns.str.startswith('loser_')]

# Replace 'winner_' with 'w_' and 'loser_' with 'l_'
new_winner_cols = winner_cols.str.replace('winner_', 'w_')
new_loser_cols = loser_cols.str.replace('loser_', 'l_')

# Create a dictionary for rename operation
rename_dict = {old: new for old, new in zip(list(winner_cols) + list(loser_cols), list(new_winner_cols) + list(new_loser_cols))}

# Rename the columns
df.rename(columns=rename_dict, inplace=True)

df.head()

,match_id,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,year,match_num,w_id,w_seed,w_entry,w_name,w_hand,w_ht,w_ioc,w_age,l_id,l_seed,l_entry,l_name,l_hand,l_ht,l_ioc,l_age,score,best_of,round,minutes,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,w_rank,w_rankpt,l_rank,l_rankpt,ATP,Location,Tournament,match_date,Series,court,Surface,Round,Best of,Winner,Loser,WRank,LRank,WPts,LPts,W1,L1,W2,L2,W3,L3,W4,L4,W5,L5,Wsets,Lsets,w_B365,l_B365,w_EX,l_EX,w_LB,l_LB,w_PS,l_PS,w_SJ,l_SJ,w_Max,l_Max,w_Avg,l_Avg,w_UB,l_UB,CBW,CBL,IWW,IWL,SBW,SBL,B&WW,B&WL,GBW,GBL,w_points,l_points,total_points,total_games,w_games,l_games,w_sets,l_sets
0,2009-339-0011,2009-339,Brisbane,Hard,32,A,2009-01-04,2009,11,103898,NaN,NaN,Julien Benneteau,R,185.00,FRA,27.00,104068,NaN,NaN,Robby Ginepri,R,183.00,USA,26.20,3-6 6-3 7-5,3,R32,127.00,10.00,9.00,99.00,62.00,41.00,17.00,15.00,2.00,6.00,8.00,2.00,88.00,53.00,32.00,21.00,15.00,4.00,9.00,40.00,1608.00,49.00,1480.00,1,Brisbane,Brisbane International,2009-01-04,ATP250,Outdoor,Hard,1st Round,3.00,Benneteau J.,Ginepri R.,40.00,49.00,1608.00,1480.00,3.00,6.00,6.00,3.00,7.00,5.00,NaN,NaN,NaN,NaN,2.00,1.00,2.10,1.66,2.25,1.65,2.10,1.66,NaN,NaN,2.10,1.73,NaN,NaN,NaN,NaN,1.96,1.80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,93.00,94.00,187.00,30,16,14,2,1
1,2009-339-0012,2009-339,Brisbane,Hard,32,A,2009-01-04,2009,12,104417,4.00,NaN,Robin Soderling,R,193.00,SWE,24.30,105023,NaN,NaN,Sam Querrey,R,198.00,USA,21.20,6-3 6-3,3,R32,55.00,7.00,2.00,52.00,30.00,26.00,13.00,9.00,0.00,0.00,4.00,3.00,49.00,27.00,20.00,9.00,9.00,2.00,5.00,17.00,2650.00,36.00,1740.00,1,Brisbane,Brisbane International,2009-01-04,ATP250,Outdoor,Hard,1st Round,3.00,Soderling R.,Querrey S.,17.00,36.00,2650.00,1740.00,6.00,3.00,6.00,3.00,NaN,NaN,NaN,NaN,NaN,NaN,2.00,0.00,1.44,2.62,1.49,2.65,1.53,2.37,NaN,NaN,1.50,2.62,NaN,NaN,NaN,NaN,1.40,2.85,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59.00,42.00,101.00,18,12,6,2,0
2,2009-339-0013,2009-339,Brisbane,Hard,32,A,2009-01-04,2009,13,104755,7.00,NaN,Richard Gasquet,R,185.00,FRA,22.50,102967,NaN,NaN,Marc Gicquel,R,188.00,FRA,31.70,4-6 6-2 6-2,3,R32,133.00,6.00,5.00,91.00,42.00,29.00,26.00,13.00,3.00,5.00,7.00,6.00,100.00,45.00,30.00,23.00,13.00,13.00,18.00,23.00,2320.00,50.00,1428.00,1,Brisbane,Brisbane International,2009-01-04,ATP250,Outdoor,Hard,1st Round,3.00,Gasquet R.,Gicquel M.,23.00,50.00,2320.00,1428.00,4.00,6.00,6.00,2.00,6.00,2.00,NaN,NaN,NaN,NaN,2.00,1.00,1.25,3.75,1.27,3.81,1.28,3.50,NaN,NaN,1.25,3.50,NaN,NaN,NaN,NaN,1.27,3.60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,102.00,89.00,191.00,26,16,10,2,1
3,2009-339-0014,2009-339,Brisbane,Hard,32,A,2009-01-04,2009,14,103758,NaN,NaN,Taylor Dent,R,188.00,USA,27.70,104327,NaN,NaN,Steve Darcis,R,178.00,BEL,24.80,7-6(0) 6-2,3,R32,86.00,8.00,5.00,68.00,37.00,31.00,13.00,10.00,6.00,8.00,4.00,2.00,70.00,46.00,24.00,14.00,10.00,1.00,5.00,865.00,28.00,59.00,1362.00,1,Brisbane,Brisbane International,2009-01-04,ATP250,Outdoor,Hard,1st Round,3.00,Dent T.,Darcis S.,865.00,59.00,28.00,1362.00,7.00,6.00,6.00,2.00,NaN,NaN,NaN,NaN,NaN,NaN,2.00,0.00,2.75,1.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.00,1.37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,76.00,62.00,138.00,21,13,8,2,0
4,2009-339-0005,2009-339,Brisbane,Hard,32,A,2009-01-04,2009,5,104269,3.00,NaN,Fernando Verdasco,L,188.00,ESP,25.10,106071,NaN,WC,Bernard Tomic,R,193.00,AUS,16.20,6-4 6-2,3,R32,65.00,1.00,2.00,50.00,39.00,31.00,5.00,9.00,3.00,4.00,3.00,1.00,56.00,34.00,19.00,9.00,9.00,9.00,13.00,15.00,2830.00,764.00,42.00,1,Brisbane,Brisbane International,2009-01-05,ATP250,Outdoor,Hard,1st Round,3.00,Verdasco F.,Tomic B.,15.00,764.00,2830.00,42.00,6.00,4.00,6.00,2.00,NaN,NaN,NaN,NaN,NaN,NaN,2.00,0.00,1.06,8.00,1.06,9.52,1.04,9.00,NaN,NaN,1.03,11.00,NaN,NaN,NaN,NaN,1.04,11.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64.00,42.00,106.00,18,12,6,2,0


In [8]:
##### REMOVE COLUMNS #####

# List of columns to be removed
cols_to_remove = [
    'tourney_id', 'tourney_date', 'match_num', 'ATP', 'Location', 'Tournament', 'Series', 
    'Surface', 'Round', 'Best of', 'Winner', 'Loser', 'WRank', 'LRank', 'Wsets', 'Lsets',
    'WPts', 'LPts', 'W1', 'L1', 'W2', 'L2', 'W3', 'L3', 'W4', 'L4', 'W5', 'L5', 
    'CBW', 'CBL', 'IWW', 'IWL', 'SBW', 'SBL', 'B&WW', 'B&WL', 'GBW', 'GBL'
]

# Remove the columns
df = df.drop(columns=cols_to_remove, axis=1)

df.head()

,match_id,tourney_name,surface,draw_size,tourney_level,year,w_id,w_seed,w_entry,w_name,w_hand,w_ht,w_ioc,w_age,l_id,l_seed,l_entry,l_name,l_hand,l_ht,l_ioc,l_age,score,best_of,round,minutes,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,w_rank,w_rankpt,l_rank,l_rankpt,match_date,court,w_B365,l_B365,w_EX,l_EX,w_LB,l_LB,w_PS,l_PS,w_SJ,l_SJ,w_Max,l_Max,w_Avg,l_Avg,w_UB,l_UB,w_points,l_points,total_points,total_games,w_games,l_games,w_sets,l_sets
0,2009-339-0011,Brisbane,Hard,32,A,2009,103898,NaN,NaN,Julien Benneteau,R,185.00,FRA,27.00,104068,NaN,NaN,Robby Ginepri,R,183.00,USA,26.20,3-6 6-3 7-5,3,R32,127.00,10.00,9.00,99.00,62.00,41.00,17.00,15.00,2.00,6.00,8.00,2.00,88.00,53.00,32.00,21.00,15.00,4.00,9.00,40.00,1608.00,49.00,1480.00,2009-01-04,Outdoor,2.10,1.66,2.25,1.65,2.10,1.66,NaN,NaN,2.10,1.73,NaN,NaN,NaN,NaN,1.96,1.80,93.00,94.00,187.00,30,16,14,2,1
1,2009-339-0012,Brisbane,Hard,32,A,2009,104417,4.00,NaN,Robin Soderling,R,193.00,SWE,24.30,105023,NaN,NaN,Sam Querrey,R,198.00,USA,21.20,6-3 6-3,3,R32,55.00,7.00,2.00,52.00,30.00,26.00,13.00,9.00,0.00,0.00,4.00,3.00,49.00,27.00,20.00,9.00,9.00,2.00,5.00,17.00,2650.00,36.00,1740.00,2009-01-04,Outdoor,1.44,2.62,1.49,2.65,1.53,2.37,NaN,NaN,1.50,2.62,NaN,NaN,NaN,NaN,1.40,2.85,59.00,42.00,101.00,18,12,6,2,0
2,2009-339-0013,Brisbane,Hard,32,A,2009,104755,7.00,NaN,Richard Gasquet,R,185.00,FRA,22.50,102967,NaN,NaN,Marc Gicquel,R,188.00,FRA,31.70,4-6 6-2 6-2,3,R32,133.00,6.00,5.00,91.00,42.00,29.00,26.00,13.00,3.00,5.00,7.00,6.00,100.00,45.00,30.00,23.00,13.00,13.00,18.00,23.00,2320.00,50.00,1428.00,2009-01-04,Outdoor,1.25,3.75,1.27,3.81,1.28,3.50,NaN,NaN,1.25,3.50,NaN,NaN,NaN,NaN,1.27,3.60,102.00,89.00,191.00,26,16,10,2,1
3,2009-339-0014,Brisbane,Hard,32,A,2009,103758,NaN,NaN,Taylor Dent,R,188.00,USA,27.70,104327,NaN,NaN,Steve Darcis,R,178.00,BEL,24.80,7-6(0) 6-2,3,R32,86.00,8.00,5.00,68.00,37.00,31.00,13.00,10.00,6.00,8.00,4.00,2.00,70.00,46.00,24.00,14.00,10.00,1.00,5.00,865.00,28.00,59.00,1362.00,2009-01-04,Outdoor,2.75,1.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.00,1.37,76.00,62.00,138.00,21,13,8,2,0
4,2009-339-0005,Brisbane,Hard,32,A,2009,104269,3.00,NaN,Fernando Verdasco,L,188.00,ESP,25.10,106071,NaN,WC,Bernard Tomic,R,193.00,AUS,16.20,6-4 6-2,3,R32,65.00,1.00,2.00,50.00,39.00,31.00,5.00,9.00,3.00,4.00,3.00,1.00,56.00,34.00,19.00,9.00,9.00,9.00,13.00,15.00,2830.00,764.00,42.00,2009-01-05,Outdoor,1.06,8.00,1.06,9.52,1.04,9.00,NaN,NaN,1.03,11.00,NaN,NaN,NaN,NaN,1.04,11.00,64.00,42.00,106.00,18,12,6,2,0


In [9]:
##### ORDER COLUMNS #####

# Define the new order of columns
new_column_order = [
    'match_id', 'match_date', 'year', 'tourney_name', 'tourney_level', 'court', 'surface', 
    'draw_size', 'round', 'best_of', 'minutes', 'total_points', 'total_games', 
    'w_points', 'l_points', 'w_games', 'l_games', 'w_sets', 'l_sets', 'score', 'w_id', 'l_id',
    'w_name', 'l_name', 'w_rank', 'l_rank', 'w_rankpt', 'l_rankpt', 'w_hand', 'l_hand', 'w_ht', 'l_ht', 
    'w_ioc', 'l_ioc', 'w_age', 'l_age', 'w_seed', 'l_seed', 'w_entry', 'l_entry', 
    'w_ace', 'l_ace', 'w_df', 'l_df', 'w_svpt', 'l_svpt', 'w_1stIn', 'l_1stIn', 'w_1stWon', 'l_1stWon',
    'w_2ndWon', 'l_2ndWon', 'w_SvGms', 'l_SvGms', 'w_bpSaved', 'l_bpSaved', 'w_bpFaced', 'l_bpFaced',
    'w_B365', 'l_B365', 'w_EX', 'l_EX', 'w_LB', 'l_LB', 'w_PS', 'l_PS', 'w_SJ', 'l_SJ',	'w_UB', 'l_UB', 
    'w_Max', 'l_Max', 'w_Avg', 'l_Avg',
]

# Get the remaining columns that were not included in the new_column_order list
remaining_columns = [col for col in df.columns if col not in new_column_order]

# Combine the two lists to get the final column order
final_column_order = new_column_order + remaining_columns

# Reorder the columns in df
df = df[final_column_order]

df.head()

,match_id,match_date,year,tourney_name,tourney_level,court,surface,draw_size,round,best_of,minutes,total_points,total_games,w_points,l_points,w_games,l_games,w_sets,l_sets,score,w_id,l_id,w_name,l_name,w_rank,l_rank,w_rankpt,l_rankpt,w_hand,l_hand,w_ht,l_ht,w_ioc,l_ioc,w_age,l_age,w_seed,l_seed,w_entry,l_entry,w_ace,l_ace,w_df,l_df,w_svpt,l_svpt,w_1stIn,l_1stIn,w_1stWon,l_1stWon,w_2ndWon,l_2ndWon,w_SvGms,l_SvGms,w_bpSaved,l_bpSaved,w_bpFaced,l_bpFaced,w_B365,l_B365,w_EX,l_EX,w_LB,l_LB,w_PS,l_PS,w_SJ,l_SJ,w_UB,l_UB,w_Max,l_Max,w_Avg,l_Avg
0,2009-339-0011,2009-01-04,2009,Brisbane,A,Outdoor,Hard,32,R32,3,127.00,187.00,30,93.00,94.00,16,14,2,1,3-6 6-3 7-5,103898,104068,Julien Benneteau,Robby Ginepri,40.00,49.00,1608.00,1480.00,R,R,185.00,183.00,FRA,USA,27.00,26.20,NaN,NaN,NaN,NaN,10.00,8.00,9.00,2.00,99.00,88.00,62.00,53.00,41.00,32.00,17.00,21.00,15.00,15.00,2.00,4.00,6.00,9.00,2.10,1.66,2.25,1.65,2.10,1.66,NaN,NaN,2.10,1.73,1.96,1.80,NaN,NaN,NaN,NaN
1,2009-339-0012,2009-01-04,2009,Brisbane,A,Outdoor,Hard,32,R32,3,55.00,101.00,18,59.00,42.00,12,6,2,0,6-3 6-3,104417,105023,Robin Soderling,Sam Querrey,17.00,36.00,2650.00,1740.00,R,R,193.00,198.00,SWE,USA,24.30,21.20,4.00,NaN,NaN,NaN,7.00,4.00,2.00,3.00,52.00,49.00,30.00,27.00,26.00,20.00,13.00,9.00,9.00,9.00,0.00,2.00,0.00,5.00,1.44,2.62,1.49,2.65,1.53,2.37,NaN,NaN,1.50,2.62,1.40,2.85,NaN,NaN,NaN,NaN
2,2009-339-0013,2009-01-04,2009,Brisbane,A,Outdoor,Hard,32,R32,3,133.00,191.00,26,102.00,89.00,16,10,2,1,4-6 6-2 6-2,104755,102967,Richard Gasquet,Marc Gicquel,23.00,50.00,2320.00,1428.00,R,R,185.00,188.00,FRA,FRA,22.50,31.70,7.00,NaN,NaN,NaN,6.00,7.00,5.00,6.00,91.00,100.00,42.00,45.00,29.00,30.00,26.00,23.00,13.00,13.00,3.00,13.00,5.00,18.00,1.25,3.75,1.27,3.81,1.28,3.50,NaN,NaN,1.25,3.50,1.27,3.60,NaN,NaN,NaN,NaN
3,2009-339-0014,2009-01-04,2009,Brisbane,A,Outdoor,Hard,32,R32,3,86.00,138.00,21,76.00,62.00,13,8,2,0,7-6(0) 6-2,103758,104327,Taylor Dent,Steve Darcis,865.00,59.00,28.00,1362.00,R,R,188.00,178.00,USA,BEL,27.70,24.80,NaN,NaN,NaN,NaN,8.00,4.00,5.00,2.00,68.00,70.00,37.00,46.00,31.00,24.00,13.00,14.00,10.00,10.00,6.00,1.00,8.00,5.00,2.75,1.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.00,1.37,NaN,NaN,NaN,NaN
4,2009-339-0005,2009-01-05,2009,Brisbane,A,Outdoor,Hard,32,R32,3,65.00,106.00,18,64.00,42.00,12,6,2,0,6-4 6-2,104269,106071,Fernando Verdasco,Bernard Tomic,15.00,764.00,2830.00,42.00,L,R,188.00,193.00,ESP,AUS,25.10,16.20,3.00,NaN,NaN,WC,1.00,3.00,2.00,1.00,50.00,56.00,39.00,34.00,31.00,19.00,5.00,9.00,9.00,9.00,3.00,9.00,4.00,13.00,1.06,8.00,1.06,9.52,1.04,9.00,NaN,NaN,1.03,11.00,1.04,11.00,NaN,NaN,NaN,NaN


In [10]:
# Export dataset to excel
df.to_excel('_data/_matches_list_v1.xlsx', index=False)